In [1]:
from google.colab import drive 
drive.mount('/gdrive')
%cd /gdrive/My Drive/Colab Notebooks/ProgettoDRV

Mounted at /gdrive
/gdrive/My Drive/Colab Notebooks/ProgettoDRV


In [2]:
import pandas as pd
import numpy as np

In [3]:
# Compute the volatility
btc = pd.read_csv("data/crypto/bitcoin_usd.csv")

In [ ]:
#btc.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-01-01,963.658020,1003.080017,958.698975,998.325012,998.325012,147775008
1,2017-01-02,998.617004,1031.390015,996.702026,1021.750000,1021.750000,222184992
2,2017-01-03,1021.599976,1044.079956,1021.599976,1043.839966,1043.839966,185168000
3,2017-01-04,1044.400024,1159.420044,1044.400024,1154.729980,1154.729980,344945984
4,2017-01-05,1156.729980,1191.099976,910.416992,1013.380005,1013.380005,510199008


In [4]:
btc.Date = pd.to_datetime(btc.Date)
btc = btc.set_index("Date")

In [5]:
# Crea una sequenza temporale senza buchi di giorni e valori
# Non c'è bisogno perchè ha già 365 valori
btc = btc.resample("D").mean()
btc = btc.fillna(method="ffill")

In [ ]:
# No null value in the dataset
#btc.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [6]:
goldETC = pd.read_csv("data/goldETC.csv")
goldETC.Date = pd.to_datetime(goldETC.Date)
goldETC = goldETC.set_index("Date")

In [ ]:
#goldETC = goldETC[goldETC.index >= pd.Timestamp("2019-01-01")]
#goldETC = goldETC[goldETC.index <= pd.Timestamp("2019-12-31")]
#goldETC["returns"] = np.log(goldETC["Close"]/goldETC["Close"].shift())

# Volatility = standard deviation of a chosen  time's window
# For annual volatility of stocks multiply by 252 
#volatility = goldETC["returns"].std()*252**.5

In [7]:
# The stock must have a 'Close' column
# The function return the volatility for a given stock as a numpy array
def volatility(stock_var, starting_year, ending_year):
  # Calcola la differenza di anni
  number_of_years = ending_year - starting_year 

  # Inizializza una lista vuota dove andrà ad inserire per ogni anno la volatilità
  vol = []

  for i in range(0, number_of_years+1):
    stock = stock_var
    stock = stock[stock.index >= pd.Timestamp(str(starting_year+i)+"-01-01")]
    stock = stock[stock.index <= pd.Timestamp(str(starting_year+i)+"-12-31")]
    # Varianza
    stock["returns"] = np.log(stock["Close"]/stock["Close"].shift())
    # Calcola la volatilià annuale
    vol.append(stock["returns"].std()*(len(stock.index))**.5)

  return np.array(vol)

In [8]:
# S&P500
sep = pd.read_csv("data/sep500.csv")
sep = sep.rename(columns={"Data":"Date"})
sep = sep.rename(columns={"Chiusura/Ultimo":"Close"})
sep.Date = pd.to_datetime(sep.Date)
sep = sep.set_index("Date")

# BundGermania
bund = pd.read_csv("data/bund10ygermania.csv")
bund = bund.rename(columns={"Data":"Date"})
bund.Date = pd.to_datetime(bund.Date, dayfirst = True)
bund = bund.set_index("Date")
bund = bund.rename(columns={"Ultimo":"Close"})
bund = bund.rename(columns={"Massimo":"Max"})
bund = bund[["Close", "Max"]]


bund["Close"] = bund["Close"].astype(str).apply(lambda x: x.replace(',','.'))
bund["Max"] = bund["Max"].astype(str).apply(lambda x: x.replace(',','.'))

# Add all the missing days in the dataset
bund = bund.resample("D").mean()

# Fill missing values of the new added days
bund = bund.fillna(method='ffill')

## here I'm adding a column with colors
import numpy as np
bund["Color"] = np.where(bund["Close"].astype(float)<0, 'red', 'green')

# Create 2 new columns where there are only positive/negative values
mask = bund['Close'] < 0
bund['Pos'] = bund['Close'].mask(mask)
bund['Neg'] = bund['Close'].mask(~mask)


In [9]:
bund_month = bund.asfreq("M", method="ffill")


In [ ]:
import plotly.graph_objects as go

import pandas as pd


# Create figure
fig = go.Figure()


#fig.add_trace(
#    go.Bar(x=list(bund.index), y=list(bund.Close), marker_color=list(bund.Color), name = "Bund"))
fig.add_trace(
    go.Bar(x=list(bund.index), y=list(bund.Pos), marker_color="green", name = "Positive bund interest"))
fig.add_trace(
    go.Bar(x=list(bund.index), y=list(bund.Neg), marker_color="red", name = "Negative bund interest"))
fig.add_trace(
    go.Scatter(x=list(bund_month.index), y=list(bund_month.Close), marker_color="rgba(0, 0, 0, 1)", name = "Bund montly trend"))


# Add range slider
fig.update_layout(
    xaxis=dict(
        #fixedrange = True,
        range= ["2019-10-22", "2020-06-10"],
        #gridcolor = "pink",
        rangeselector=dict(
            # Colore dei bottoni quando ci passi sopra con mosue
            #activecolor = "red",
            # Colore dei bottoni
            #bgcolor = "yellow",
            #Posizione dei bottoni sull'asse x (valori tra -2 e 3)
            x = 0.5,
            buttons=list([
                dict(count=6,
                     label="6 months",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="1 year",
                     step="year",
                     stepmode="backward"),
                dict(count=3,
                     label="3 year",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

# Add shapes
fig.update_layout(
    shapes=[
        dict(
            # Covid-19 period shape
            fillcolor="rgba(135,206,250, 0.3)",
            line={"width": 0},
            type="rect",
            x0="2020-01-01",
            x1="2020-03-10",
            xref="x",
            y0=0.02,
            y1=0.98,
            yref="paper"
        ),
          dict(
            # Positive (green) shadow
            fillcolor="rgba(0, 255, 0, 0.1)",
            line={"width": 0},
            type="rect",
            xref="x",
            x0="2015-01-2",
            x1="2022-04-14",
            y0=0.42,
            y1=1,
            yref="paper"
        ),
        dict(
            # Negative (red) shadow
            fillcolor="rgba(255 , 0, 0, 0.1)",
            line={"width": 0},
            type="rect",
            xref="x",
            x0="2015-01-2",
            x1="2022-04-14",
            y0=0.42,
            y1=0,
            yref="paper"
        )
    ]
)

# Text
fig.add_annotation(x= "2020-01-01", y=0.98,
            text="Covid start spreading",
         borderwidth=1,
         borderpad=4,
        bgcolor="lightblue",
        opacity=0.9
)

fig.update_layout(
    # Modifica titolo
    title={
        'text': "Bund 10Y interests",
        'y':0.95,
        'x':0.5,
        "font.size": 20,
        'xanchor': 'center',
        'yanchor': 'top'}
)

fig.update_layout(legend=dict(
    yanchor="top",
    y=1,
    xanchor="right",
    x=0.2,
    borderwidth = 1,
    bordercolor = "royalblue",
    bgcolor = "#e6f9ff"
))

fig.show()
fig.write_html("./grafici/bund_tedesco.html")

In [10]:
# Calcola la volatilità dell'oro
gold_vol = (volatility(goldETC, 2018, 2021)*100).round(2)
btc_vol = (volatility(btc, 2018, 2021)*100).round(2)
sep_vol = (volatility(sep, 2018, 2021)*100).round(2)

# Crea un dataframe per gli anni 2019-2021
volatilities = pd.DataFrame(data = {'year': ["2018", "2019", "2020", "2021"]})

# Aggiunge la colonna riguardante l'oro
volatilities["Gold"] = gold_vol.tolist()
volatilities["Bitcoin"] = btc_vol.tolist()
volatilities["Sep"] = sep_vol.tolist()
volatilities = volatilities.set_index("year")

In [ ]:
import plotly.graph_objects as go


fig = go.Figure(data=[
    go.Bar(name='Bitcoin', x= volatilities.index, y=volatilities.Bitcoin, text = volatilities.Bitcoin, marker = dict(color ="orange")),
    go.Bar(name='S&P500', x= volatilities.index, y=volatilities.Sep, text = volatilities.Sep, marker = dict(color ="green")),
    go.Bar(name='Gold', x= volatilities.index, y=volatilities.Gold, text = volatilities.Gold, marker = dict(color ="gold"))

])
# Change the bar mode
fig.update_layout(barmode='group')
fig.update_traces(texttemplate='%{text:.3s}', textposition='outside')

fig.update_layout(
    # Modifica titolo
    title={
        'text': "Volatilities compared",
        'y':0.95,
        'x':0.5,
        "font.size": 25,
        'xanchor': 'center',
        'yanchor': 'top'}
)

# Adds a rectangle 
fig.add_shape(
    dict(type="rect", x0=1.5, y0=0, x1=2.5, y1=85, line=dict(width=4)),line_color="red",  
)
fig.add_annotation(x=1.45, y=85,
            text="Covid year",
            showarrow=True,
            arrowhead=2,
            arrowwidth=3,
           arrowcolor="red",
           bordercolor="#c7c7c7",
         borderwidth=1,
         borderpad=4,
        bgcolor="red",
        opacity=0.9
)
fig.show()
fig.write_html("./grafici/comparison_volatilities.html")

In [11]:
# Volatilità generale del bitcoin dal 2015
btc = btc[btc.index >= pd.Timestamp("2015-01-01")]
btc_vol = (volatility(btc, 2015, 2021)*100).round(2)
# Crea un dataframe per gli anni 2019-2021
volatilities = pd.DataFrame(data = {'year': ["2015","2016","2017","2018", "2019", "2020", "2021"]})
volatilities["Bitcoin"] = btc_vol.tolist()
volatilities = volatilities.set_index("year")

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

fig = px.bar(volatilities, x= volatilities.index, y="Bitcoin",
             hover_data=['Bitcoin'], color='Bitcoin', labels={'Bitcoin':'Volatility'},  
             # Aggiunge il simbolo % dopo i numeri 
             text=volatilities.Bitcoin.apply(lambda x: '{0:1.2f}%'.format(x) ))

fig.update_traces(textposition='outside')

fig.add_trace(go.Scatter(x= volatilities.index[:2], y= volatilities.Bitcoin[:2], mode = "lines+markers", name = "", showlegend = False,
                          line = dict(color='green', width=5 )))
fig.add_trace(go.Scatter(x= volatilities.index[1:3], y= volatilities.Bitcoin[1:3], mode = "lines+markers", name = "", showlegend = False,
                          line = dict(color='red', width=5 )))
fig.add_trace(go.Scatter(x= volatilities.index[2:5], y= volatilities.Bitcoin[2:5], mode = "lines+markers", name = "", showlegend = False,
                          line = dict(color='green', width=5 )))
fig.add_trace(go.Scatter(x= volatilities.index[4:7], y= volatilities.Bitcoin[4:7], mode = "lines+markers", name = "", showlegend = False,
                          line = dict(color='red', width=5 )))


fig.update_layout(
    autosize=False,
    # Modifica dimensioni figura
    width=1200,
    height=600,
    # Modifica dimensione testo asse y
    yaxis=dict(
        titlefont=dict(size=20),
    ),
    # Modifica titolo
    title={
        'text': "Bitcoin annual volatility history from 2015 to 2021",
        'y':0.95,
        'x':0.5,
        "font.size": 20,
        'xanchor': 'center',
        'yanchor': 'top'}
)



fig.show()
fig.write_html("./grafici/btc_year_volatility.html")